Reading in packages that will we used later

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [2]:
#Adjusting batch size to 32 and reducing img size from 180x180 to see if this improves model fitting performance
data_dir= "C:\\Users\\amanda.persson\\OneDrive - Accenture\\Private\\ASL images\\asl_alphabet_train\\asl_alphabet_train"
batch_size = 8
img_height = 120
img_width = 120



Reading all test data for later use

In [3]:
#Intentionally manipulated to provide less data for testing that model works
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels="inferred", 
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 87000 files belonging to 29 classes.
Using 69600 files for training.


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels="inferred",  
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Defining the model that will later be trained

In [6]:
num_classes = 29

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),  
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),  
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 120, 120, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 120, 120, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 120, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 60, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 60, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
8700/8700 [==============================] - 1756s 194ms/step - loss: 3.1481 - accuracy: 0.0888 - val_loss: 15.6530 - val_accuracy: 0.0927
Epoch 2/10
8700/8700 [==============================] - 1531s 176ms/step - loss: 1.8356 - accuracy: 0.4018 - val_loss: 1.3179 - val_accuracy: 0.6121
Epoch 3/10
8700/8700 [==============================] - 1466s 169ms/step - loss: 0.6733 - accuracy: 0.7727 - val_loss: 0.6095 - val_accuracy: 0.8010
Epoch 4/10
8700/8700 [==============================] - 1454s 167ms/step - loss: 0.2331 - accuracy: 0.9232 - val_loss: 0.1749 - val_accuracy: 0.9390
Epoch 5/10
8700/8700 [==============================] - 1434s 165ms/step - loss: 0.1408 - accuracy: 0.9564 - val_loss: 0.7671 - val_accuracy: 0.8295
Epoch 6/10
8700/8700 [==============================] - 1429s 164ms/step - loss: 0.0947 - accuracy: 0.9707 - val_loss: 0.3423 - val_accuracy: 0.9060
Epoch 7/10
8700/8700 [==============================] - 1465s 168ms/step - loss: 0.0703 - accuracy: 0.978

Check accuracy of trained model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("3Convlayer_20210314_batch_norm")